# Cats vs Dogs Image Classification

In [1]:
#classifying images of dogs and cats using deep learning with keras 

In [2]:
#importing needed libraries
from keras.models import Sequential
from keras.layers import Conv2D,Activation,MaxPooling2D,Dense,Flatten,Dropout
import numpy as np

Using TensorFlow backend.


In [3]:
#Initializing the CNN
classifier = Sequential()

In [4]:
#Adding convolutional layer which does filtering
classifier.add(Conv2D(32,(3,3),input_shape=(64,64,3))) 
#32 is the no.of filters. the filter is an array of numbers, (3,3) is the size of the filter.
#The input image is 64*64*3 size #64 height and 64 width and 3 refers RGB values
#Each of the numbers in this array(64,64,3) is given values from 0 to 255 
#which describes the pixel intensity at that point. 
#The output of this layer will be some feature maps

In [5]:

#Let's pass the feature maps through an activation layer called ReLu
classifier.add(Activation('relu'))
#ReLu replaces all the negative pixel values in the feature map with 0.

In [6]:
#Adding pooling layer
classifier.add(MaxPooling2D(pool_size =(2,2)))
#Pooling reduces the dimensionality of each feature map, but retains the most imp. info.
#This reduces the computational complexity of our network
#Here, we used maxpooling with 2*2 filter. the filter will take the max values from each pool.

In [7]:
#A classic CNN has 3 Convolutional blocks followed by a fully connected layer.
#we created the first set of layers. We can repeat this twice more.
classifier.add(Conv2D(32,(3,3))) 
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [8]:
#One more....
classifier.add(Conv2D(64,(3,3))) 
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [9]:
#To prevent overfitting, we use dropout layer.
#This layer drops out a random set of activations in that layer by setting them to zero as data flows through it.
#To prepare our model for droupot, we first flatten the feature map to 1D.
classifier.add(Flatten())
#Then we want to initialize a fully connected nw by Dense function
classifier.add(Dense(64))
# and apply relu to it.
classifier.add(Activation('relu'))
#Add the Dropout layer
classifier.add(Dropout(0.5))

In [10]:
#After dropout, we'll initialize 1 more fully connected layer.
#This will output an n D vector, where n is the no.of classes we have, that is 2.
classifier.add(Dense(1))

In [11]:
#By applying a sigmoid to it, it'll convert the data to probabilities for each class.
classifier.add(Activation('sigmoid'))

In [12]:
#compiling cnn
classifier.compile(optimizer ='rmsprop',
                   loss ='binary_crossentropy',
                   metrics =['accuracy'])
#Optimizer rmsprop will perform gradient descent(finding the min. of a function)
#binary_crossentropy is the prefered loss fn for binary classif. problems
#metrics is set to accuracy

In [13]:
#reducing overfitting by data augmentation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale =1./255,
                                   shear_range =0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip =True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
#Setting Train and Test directories
training_set = train_datagen.flow_from_directory('C:/Users/Lab/Desktop/CatDogData/train',
                                                target_size=(64,64),
                                                batch_size= 32,
                                                class_mode='binary')
#and..
test_set = test_datagen.flow_from_directory('C:/Users/Lab/Desktop/CatDogData/test',
                                           target_size = (64,64),
                                           batch_size = 32,
                                           class_mode ='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [15]:
from IPython.display import display
from PIL import Image


classifier.fit_generator(training_set,
                        steps_per_epoch =20000,
                        epochs = 2,
                        validation_data =test_set,
                        validation_steps = 5000)

Epoch 1/2
20000/20000 [==============================] - 10341s 517ms/step - loss: 0.4067 - acc: 0.8184 - val_loss: 0.3150 - val_acc: 0.8641
Epoch 2/2
20000/20000 [==============================] - 15417s 771ms/step - loss: 0.3739 - acc: 0.8445 - val_loss: 0.3458 - val_acc: 0.8683


In [16]:
from keras.models import load_model 
classifier.save('catdog_cnn_model.h5') # creates a HDF5 file 'catdog_model.h5'

In [42]:
#loading the saved model
from keras.models import load_model 
classifier = load_model('catdog_cnn_model.h5')

#testing
import numpy as np
from keras.preprocessing import image
img = image.load_img('C:/Users/Lab/Desktop/CatDogData/predict/d3.jpeg', target_size=(64,64))
img = np.array(img)
img =np.expand_dims(img, axis =0)
prediction = classifier.predict(img)
print(prediction)

[[1.]]


In [ ]:
#testing
import numpy as np
from keras.preprocessing import image
test_image =image.load_img('C:/Users/Lab/Desktop/CatDogData/predict/d3.jpeg',target_size =(64,64))
test_image =image.img_to_array(test_image)
test_image =np.expand_dims(test_image, axis =0)
result = classifier.predict(test_image)
if result[0][0] >= 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)